In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()

# url = "https://schedulebuilder.yorku.ca/vsb/criteria.jsp?access=0&lang=en&tip=0&page=results&scratch=0&term=2023102119"
# course_name = "LE-EECS-2101-3.00-EN"
# course_term = "Term W"
# course_section = "Section N"

url = "https://w2prod.sis.yorku.ca/Apps/WebObjects/cdm"
course_name = "Fundamentals of Data Structures"
course_cat = "G26C01"
username = os.getenv("USER_NAME")
password = os.getenv("PASSWORD")

In [3]:
try:
    service = Service(executable_path="chromedriver.exe")
    options = Options()
    # options.add_argument("--headless")
    options.add_argument("start-maximized")
    driver = webdriver.Chrome(service=service, options=options)
except KeyboardInterrupt as e:
    print("exiting...")
except Exception as e:
    print(e)


In [4]:
def get_info():
    info = {}

    try:
        # Search for the course
        driver.find_element(By.ID, "code_number").send_keys(course_name + Keys.ENTER)

        # Select term
        term = driver.find_element(By.CSS_SELECTOR, "#requirements > div:nth-child(3) > div.courseDiv.bc1.bd1 > div:nth-child(12) > select")
        term.click()
        terms = term.text.split("\n")
        term.find_elements(By.TAG_NAME, "option")[terms.index(course_term)].click()

        # Select section
        section = driver.find_element(By.CSS_SELECTOR, "#requirements > div:nth-child(3) > div.courseDiv.bc1.bd1 > div:nth-child(13) > select")
        section.click()
        sections = section.text.split("\n")
        section.find_elements(By.TAG_NAME, "option")[sections.index(course_section)].click()

        # Get the course information
        courses = driver.find_elements(By.CLASS_NAME, "course_box")

        # TODO: Labs and tutorials seats might not all be avaialable

        for course in courses:
            lines = course.text.split("\n")
            title = lines[0]
            catalog = lines[5]
            seats = lines[6].split(" ")[-1] == "Available"
            prof = lines[-1]
            if title.startswith(course_name):
                info = {
                    "title": title,
                    "catalog": catalog,
                    "seats": seats,
                    "prof": prof
                }
    except Exception as e:
        sleep(5)
        # print("Waiting for page to load...")
        print(e)

    return info

def get_info_2():
    try:
        # Going to the course title search
        driver.get(url)
        driver.find_element(By.XPATH, "/html/body/p/table/tbody/tr[2]/td[1]/table/tbody/tr[5]/td/a/img").click()

        # Clear the search bar
        driver.find_element(By.ID, "courseTitleInput").clear()
        
        # Search for the course
        driver.find_element(By.ID, "courseTitleInput").send_keys(course_name + Keys.ENTER)

        # Get the course url
        course_url = driver.find_element(By.CSS_SELECTOR, "body > table > tbody > tr:nth-child(2) > td:nth-child(2) > table > tbody > tr:nth-child(2) > td > table > tbody > tr > td > table:nth-child(3) > tbody > tr:nth-child(2) > td:nth-child(3) > a").get_attribute("href")
        driver.get(course_url)

        row = driver.find_element(By.CSS_SELECTOR, "body > table > tbody > tr:nth-child(2) > td:nth-child(2) > table > tbody > tr:nth-child(2) > td > table > tbody > tr > td > table:nth-child(9) > tbody").find_elements(By.TAG_NAME, "tr")[0]
        login_url = row.find_elements(By.TAG_NAME, "td")[0].find_element(By.TAG_NAME, "table").find_element(By.TAG_NAME, "tbody").find_elements(By.TAG_NAME, "tr")[1].find_elements(By.TAG_NAME, "td")[0].find_element(By.TAG_NAME, "a").get_attribute("href")

        driver.get(login_url)

        # Login
        driver.find_element(By.ID, "mli").send_keys(username)
        driver.find_element(By.ID, "password").send_keys(password + Keys.ENTER)

        # Wait for the duo mobile iframe to load
        sleep(3)

        # Wait for the duo mobile iframe to load
        WebDriverWait(driver, 5).until(
            EC.frame_to_be_available_and_switch_to_it((By.ID, "duo_iframe"))
        )

        # Click on send me a push
        driver.find_element(By.CSS_SELECTOR, "#auth_methods > fieldset > div.row-label.push-label > button").click()
        driver.find_elements(By.TAG_NAME, "button")[0].click()

        ####################
        # Wait for the page to load
        ####################
        sleep(20)
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/table/tbody/tr[2]/td[2]/table/tbody/tr[2]/td/table/tbody/tr/td/table[2]/tbody"))
        )

        # Scraping for info
        for el in driver.find_element(By.XPATH, "/html/body/table/tbody/tr[2]/td[2]/table/tbody/tr[2]/td/table/tbody/tr/td/table[2]/tbody").find_elements(By.TAG_NAME, "tr"):
            if course_cat in el.text:
                lines = el.text.split("\n")
                prof = lines[-2]
                if el.text.find("Section/Course") != -1 or el.text.find("Seats Available") != -1:
                    seats = lines[1].split(" ")[-1]
                else:
                    seats = "Available"
                info = {
                    "seats": seats,
                    "prof": prof
                }
                break

        return info

    except Exception as e:
        print(e)

In [5]:
print(get_info_2())

Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/table/tbody/tr[2]/td[2]/table/tbody/tr[2]/td/table/tbody/tr/td/table[2]/tbody"}
  (Session info: chrome=120.0.6099.110); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6CE604D02+56194]
	(No symbol) [0x00007FF6CE5704B2]
	(No symbol) [0x00007FF6CE4176AA]
	(No symbol) [0x00007FF6CE4616D0]
	(No symbol) [0x00007FF6CE4617EC]
	(No symbol) [0x00007FF6CE4A4D77]
	(No symbol) [0x00007FF6CE485EBF]
	(No symbol) [0x00007FF6CE4A2786]
	(No symbol) [0x00007FF6CE485C23]
	(No symbol) [0x00007FF6CE454A45]
	(No symbol) [0x00007FF6CE455AD4]
	GetHandleVerifier [0x00007FF6CE97D5BB+3695675]
	GetHandleVerifier [0x00007FF6CE9D6197+4059159]
	GetHandleVerifier [0x00007FF6CE9CDF63+4025827]
	GetHandleVerifier [0x00007FF6CE69F029+687785]
	(No symbol) [0x00007FF6CE57B508]
	(No symbol) [0x00